### 練習問題2.73
[2.3.2節](../texts/2.3.2.ipynb)では、記号微分を⾏うプログラムについて説明した。

    (define (deriv exp var)
      (cond ((number? exp) 0)
            ((variable? exp)
              (if (same-variable? exp var) 1 0))
                 ((sum? exp)
                  (make-sum (deriv (addend exp) var)
                  (deriv (augend exp) var)))
           ((product? exp)
             (make-sum (make-product
                (multiplier exp)
                  (deriv (multiplicand exp) var))
                (make-product
                  (deriv (multiplier exp) var)
                    (multiplicand exp))))
           ⟨more rules can be added here⟩
           (else (error "unknown expression type:
             DERIV" exp))))

このプログラムは、微分する式の型によってディスパッチを実⾏していると捉えることもできる。この場合、データの "タイプタグ" は代数演算記号 (+ など) で、
⾏う演算は deriv ということになる。
基本的な微分を⾏う⼿続きを次のように書き直すと、
プログラムをデータ主導スタイルに変形できる。

    (define (deriv exp var)
      (cond ((number? exp) 0)
            ((variable? exp) (if (same-variable? exp var) 1 0))
                              (else ((get 'deriv (operator exp))
                                    (operands exp) var))))

    (define (operator exp) (car exp))

    (define (operands exp) (cdr exp))

  - a. 上で何をしているか説明せよ。⼿続き number? と variable?は、なぜデータ主導ディスパッチとして取り込むことができないのだろうか。

  - b. 和と積に対する微分⼿続きと、上記のプログラムで使っているテーブルにそれらを組み込む補助コードを書け。

  - c. 任意の微分規則 (例えば、[練習問題2.56](2.56.ipynb)の指数の微分など) を選び、それをこのデータ主導システムに組み込め。

  - d. この単純な代数操作では、式の型は式をまとめる代数演算⼦となっている。しかし、仮に⼿続きのインデックスを逆にして、deriv でディスパッチを⾏う箇所を次のようにするとする。

        ((get (operator exp) 'deriv) (operands exp) var)


    これに対応して、微分システムにはどのような変更を加える
    必要があるだろうか。

In [47]:
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
                          (else ((get 'deriv (operator exp))
                                (operands exp) var))))

(define (operator exp) (car exp))

(define (operands exp) (cdr exp))

In [48]:
(define (lookup key table)
  (let ((record (assoc key (cdr table))))
    (if record (cdr record)
        #f)
    )
  )

(define (assoc key records)
  (cond ((null? records) #f)
        ((equal? key (caar records)) (car records))
        (else (assoc key (cdr records)))
        )
  )

(define (insert! key value table)
  (let ((record (assoc key (cdr table))))
    (if record (set-cdr! record value)
        (set-cdr! table (cons (cons key value) (cdr table)))
        )
    )
  'ok)

(define (lookup key-1 key-2 table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (cdr record)
              #f
              )
          )
        #f
        )
    )
  )

(define (insert! key-1 key-2 value table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (set-cdr! record value)
              (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
              )
          )
        (set-cdr! table (cons (list key-1 (cons key-2 value)) (cdr table)))
        )
    )
  'ok)

(define (make-table)
  (let ((local-table (list '*table*)))
    (define (lookup key-1 key-2)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (cdr record) #f))
            #f)
        )
      )
    (define (insert! key-1 key-2 value)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (set-cdr! record value)
                  (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
                  )
              )
            (set-cdr! local-table (cons (list key-1 (cons key-2 value)) (cdr local-table )))
            )
        )
      'ok)
    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
            ((eq? m 'insert-proc!) insert!)
            (else (error "Unknown operation: TABLE" m)))
      )
    dispatch)
  )

;(define (make-table) (list '*table*))

(define operation-table (make-table))
(define get (operation-table 'lookup-proc))
(define put (operation-table 'insert-proc!))

In [53]:
; eは変数か?
(define (variable? x)
  (symbol? x)) 

; v1とv2は同じ変数か?
(define (same-variable? v1 v2)
  (and (variable? v1) (variable? v2) (eq? v1 v2)))

; eは和か?
(define (sum? x)
  (and (pair? x) (eq? (car x) '+)))

; 和eの加数
(define (addend s)
  ;(cadr s)
  (car s)
  ) 

; 和eの被加数
(define (augend s)
  ;(caddr s)
  (cadr s)
  )

; a1とa2の和を構築する
(define (make-sum a1 a2)
  (list '+ a1 a2))

;eは積か?
(define (product? x)
  (and (pair? x) (eq? (car x) '*))) 

; 積eの乗数
(define (multiplier p)
  ;(cadr p)
  (car p)
  ) 

;積eの被乗数
(define (multiplicand p)
  ;(caddr p)
  (cadr p)
  )

; m1とm2の積を構築する
(define (make-product m1 m2)
  (list '* m1 m2))

(define (make-sum a1 a2)
  (cond ((=number? a1 0) a2)
        ((=number? a2 0) a1)
        ((and (number? a1) (number? a2)) (+ a1 a2))
        (else (list '+ a1 a2)))
  )
(define (=number? exp num)
  (and (number? exp) (= exp num)))
(define (make-product m1 m2)
  (cond ((or (=number? m1 0) (=number? m2 0)) 0)
        ((=number? m1 1) m2) ((=number? m2 1) m1)
        ((and (number? m1) (number? m2)) (* m1 m2))
        (else (list '* m1 m2)))
  )

In [54]:
(put 'deriv '+
     (lambda (exp var)(make-sum (deriv (addend exp) var) (deriv (augend exp) var))
        )
     )

ok

In [55]:
(put 'deriv '*
     (lambda (exp var)
       (make-sum (make-product (multiplier exp)
                                                (deriv (multiplicand exp) var))
                                  (make-product (deriv (multiplier exp) var) (multiplicand exp)))
       )
     )

ok

In [56]:
(deriv '(+ x 3) 'x)

1

In [57]:
(deriv '(* x y) 'x)

y

In [58]:
(deriv '(* (* x y) (+ x 3)) 'x)

(+ (* x y) (* y (+ x 3)))

In [63]:

; べき乗の基数
(define (base x)
  ;(cadr x)
  (car x)
  )

; 指数
(define (exponent x)
  ;(caddr x)
  (cadr x)
  )

; べき数を構築する
(define (make-exponentiation m1 m2)
  (cond ((=number? m2 0) 1)
        ((=number? m2 1) m1)
        ;((=number? m1 1) m2) ((=number? m2 1) m1)
        ((and (number? m1) (number? m2) (> m2 0)) ((make-exponentiation (* m1 m1) (- m2 1))))
        (else (list '** m1 m2)))
  )

(put 'deriv '**
     (lambda (exp var)
         (let ((u (base exp))
               (e (exponent exp)))
            (make-product (make-product e (make-exponentiation u (- e 1))) (deriv u var))
           )
       )
     )


ok

In [64]:
(deriv '(** x 3) 'x)

(* 3 (** x 2))

In [65]:
(deriv '(** (+ x 1) 2) 'x)

(* 2 (+ x 1))

In [67]:
(define (deriv exp var)
  (cond ((number? exp) 0)
        ((variable? exp) (if (same-variable? exp var) 1 0))
                          (else ((get (operator exp) 'deriv)
                                (operands exp) var))))

(put '+ 'deriv
     (lambda (exp var)(make-sum (deriv (addend exp) var) (deriv (augend exp) var))
        )
     )

(put '* 'deriv
     (lambda (exp var)
       (make-sum (make-product (multiplier exp)
                                                (deriv (multiplicand exp) var))
                                  (make-product (deriv (multiplier exp) var) (multiplicand exp)))
       )
     )

(put '** 'deriv
     (lambda (exp var)
         (let ((u (base exp))
               (e (exponent exp)))
            (make-product (make-product e (make-exponentiation u (- e 1))) (deriv u var))
           )
       )
     )


ok

In [68]:
(deriv '(+ x 3) 'x)

1

In [69]:
(deriv '(* x y) 'x)

y

In [70]:
(deriv '(* (* x y) (+ x 3)) 'x)

(+ (* x y) (* y (+ x 3)))

In [71]:
(deriv '(** x 3) 'x)

(* 3 (** x 2))